In [ ]:
!pip install -q datasets
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.9 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `h

In [ ]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('vibhorag101/phr_mental_therapy_dataset')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1))

def preprocess_data(entry):
    conversation = entry['text']

    # Extract system prompt and all following content
    system_prompt_pattern = re.compile(r'\[INST\] <<SYS>>(.*?)<</SYS>(.*)', re.DOTALL)
    system_prompt_match = system_prompt_pattern.search(conversation)
    system_prompt = system_prompt_match.group(1).strip()
    remaining_conversation = system_prompt_match.group(2)
    # print("System prompt extracted successfully.", system_prompt)

    # Now process the remaining_conversation for user-bot exchanges
    exchanges_pattern = re.compile(r'\[INST\](.*?)\[/INST\](.*?)(?=\[INST\]|<\/s>|$)', re.DOTALL)
    segments = remaining_conversation.split('</s><s>')
    processed_data = []

    for index, segment in enumerate(segments):
        # Prepend [INST] if the segment does not start with it and is not empty
        if not segment.startswith('[INST]') and segment.strip():
            segment = '[INST]' + segment
        # print(f"Processing segment {index}: {segment}")

        matches = exchanges_pattern.findall(segment)
        # print(f"Found {len(matches)} matches in segment {index}.")
        for user_input, bot_response in matches:
            formatted_segment = f"<s>[INST] <<SYS>> {system_prompt} </SYS>> {user_input.strip()} [/INST] {bot_response.strip()} </s>"
            processed_data.append(formatted_segment)

    # Return the list of processed data instead of printing
    return processed_data

    # Output the processed data
    # for data in processed_data:
    #     print(data)

# List to hold all processed entries
processed_entries = []

# Iterate through each entry in the subset
for entry in dataset:
    processed_entries.extend(preprocess_data(entry))

In [ ]:
# Now print all processed entries
for processed_entry in processed_entries:
    print(processed_entry)

<s>[INST] <<SYS>> You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. </SYS>> Hi, I hope you're doing well today. I just wanted to talk to someone about how I've been feeling lately. I've been feeling frustrated with my health. [/INST] Hello ! I'm here to listen and support you. It's great that you reached out. Can you tell me more about what's been going on with your health that's been causing frustration? </s>
<s>[INST] <<SYS>> You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerf

In [ ]:
from datasets import load_dataset

# Load the dataset from Hugging Face Hub
dataset = load_dataset('Amod/mental_health_counseling_conversations')

# Function to format the dataset entries
def format_entry(entry):
    formatted = f"<s>[INST] <<SYS>> You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information. </SYS>> {entry['Context']} [/INST] {entry['Response']} </s>"
    return {'Formatted': formatted}

# Apply the formatting function to the dataset
formatted_dataset = dataset.map(format_entry)

# Show a formatted example
print(formatted_dataset)

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response', 'Formatted'],
        num_rows: 3512
    })
})


In [ ]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('vibhorag101/phr_mental_therapy_dataset')

# Slice the dataset
dataset = dataset['train'].select(range(125,130))

def preprocess_data(entry):
    # Initialize list to hold processed user inputs
    processed_data = []

    # Regex to capture user input within [INST]...[/INST] or following <</SYS>>...[/INST]
    pattern = re.compile(r'\[INST\](.*?)\[/INST\]|<</SYS>>(.*?)\[/INST\]', re.DOTALL)

    # Iterate over each entry in the dataset
    for entry in dataset:
        # Get the conversation from the dataset entry
        conversation = entry['text']

        # Find all occurrences of user inputs in the conversation
        matches = pattern.findall(conversation)
        for match in matches:
            # Each match is a tuple, where one group will be empty depending on the match
            user_input = match[0] if match[0] else match[1]
            processed_data.append(user_input.strip())

    return processed_data

# List to hold all processed entries
processed_entries = []

# Iterate through each entry in the subset
for entry in dataset:
    processed_entries.extend(preprocess_data(entry))

In [ ]:
# Now print all processed entries
for processed_entry in processed_entries:
    print(processed_entry)

<<SYS>>
You are a helpful and joyous mental therapy assistant. Always answer as helpfully and cheerfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>

I've been feeling so sad and overwhelmed lately. Work has become such a massive source of stress for me.
I recently got a promotion at work, which I thought would be exciting. But the added responsibilities and pressure have just taken a toll on my mental health. It's been a really moving experience for me.
Well, the workload has increased significantly, and I find it hard to maintain a work-life balance. I've been staying late at the office, and it fee

In [ ]:
# Function to format the dataset entries
def format_entry(entry):
    # Define the system prompt
    system_prompt = ("<<SYS>> You are a helpful and joyous mental therapy assistant. Always answer as helpfully "
                       "and cheerfully as possible, while being safe. Your answers should not include any harmful, "
                       "unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your "
                       "responses are socially unbiased and positive in nature. If a question does not make any sense, "
                       "or is not factually coherent, explain why instead of answering something not correct. If you "
                       "don't know the answer to a question, please don't share false information. <</SYS>")

    # Check if the entry contains a system prompt and replace it
    if "<<SYS>>" in entry:
        # Remove everything from <<SYS>> to <</SYS>>
        entry = re.sub(r'<<SYS>>.*?<</SYS>', '', entry, flags=re.DOTALL).strip()
        # Prepend the system prompt
        formatted = f"{system_prompt} <s>[INST] {entry} [/INST]"
    else:
        # If no system prompt, prepend the system prompt directly
        formatted = f"{system_prompt} <s>[INST] {entry} [/INST]"

    return formatted

# Apply the formatting function to each entry in the list
formatted_entries = [format_entry(entry) for entry in processed_entries]

# Print formatted entries
# for formatted_entry in formatted_entries:
#     print(formatted_entry)
#     print()

In [ ]:
from datasets import Dataset, DatasetDict
processed_dataset = Dataset.from_dict({'text': formatted_entries})
print(processed_dataset)

Dataset({
    features: ['text'],
    num_rows: 860
})


In [ ]:
# from datasets import load_dataset
# import re

# # Load the dataset
# dataset = load_dataset('vibhorag101/phr_mental_therapy_dataset')

# # Shuffle the dataset and slice it
# # dataset = dataset['train'].shuffle(seed=42).select(range(1))

# # Select the first row of the dataset
# dataset = dataset['train'].select([0])

# def preprocess_data(entry):
#     conversation = entry['text']

#     # Extract system prompt and all following content
#     system_prompt_pattern = re.compile(r'\[INST\] <<SYS>>(.*?)<</SYS>(.*)', re.DOTALL)
#     system_prompt_match = system_prompt_pattern.search(conversation)
#     if system_prompt_match:
#         system_prompt = system_prompt_match.group(1).strip()
#         remaining_conversation = system_prompt_match.group(2)
#         print("System prompt extracted successfully.", system_prompt)
#         print("Remaining conversation:", remaining_conversation)  # Print first 500 chars of remaining conversation for debugging
#     else:
#         print("Failed to extract system prompt.")
#         return

#     # Now process the remaining_conversation for user-bot exchanges
#     exchanges_pattern = re.compile(r'\[INST\](.*?)\[/INST\](.*?)(?=\[INST\]|<\/s>|$)', re.DOTALL)
#     segments = remaining_conversation.split('</s><s>')
#     processed_data = []

#     for index, segment in enumerate(segments):
#         # Prepend [INST] if the segment does not start with it and is not empty
#         if not segment.startswith('[INST]') and segment.strip():
#             segment = '[INST]' + segment
#         print(f"Processing segment {index}: {segment}")  # Debug output

#         matches = exchanges_pattern.findall(segment)
#         if matches:
#             print(f"Found {len(matches)} matches in segment {index}.")
#             for user_input, bot_response in matches:
#                 formatted_segment = f"<s>[INST] <<SYS>> {system_prompt} </SYS>> {user_input.strip()} [/INST] {bot_response.strip()} </s>"
#                 processed_data.append(formatted_segment)
#         else:
#             print("No matches found in this segment:", segment)

#     # Output the processed data
#     if processed_data:
#         for data in processed_data:
#             print(data)
#     else:
#         print("No processed data to print.")

# # Assuming first_row_dataset[0] is accessible
# preprocess_data(dataset[0])